In [6]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# Define your transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

''' for fruits-360 dataset with preslit data
# Load your dataset
train_dataset = datasets.ImageFolder(root='fruits-360/Training', transform=data_transforms)
val_dataset = datasets.ImageFolder(root='fruits-360/Test', transform=data_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)'''


# for custom dataset
# Load the full dataset
train_dataset = datasets.ImageFolder(root='RealWaste', transform=data_transforms)

# Split the dataset
train_size = int(0.8 * len(train_dataset))  # 80% for training
test_size = len(train_dataset) - train_size  # 20% for testing
train_dataset, eval_dataset = random_split(train_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

# Get the number of classes
num_classes = len(train_dataset.dataset.classes)

# Print the number of classes
print('Number of classes:', num_classes)


Number of classes: 9


In [7]:
import torchvision.models as models
import torch.nn as nn
model = models.efficientnet_b0(pretrained=True)  # Assuming this is available

# Freeze the model parameters
for param in model.parameters():
	param.requires_grad = False


# Modify the classifier for your dataset
num_ftrs = model.classifier[1].in_features
# add 


model.classifier = nn.Sequential(
	nn.Flatten(),
	nn.Linear(num_ftrs, num_classes)
)

print(model.named_parameters)

<bound method Module.named_parameters of EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
   

In [13]:
 #Calculate the number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Calculate the number of non-trainable parameters
non_trainable_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)

# Print the results
print(f'Total parameters: {trainable_params + non_trainable_params}')
print(f'Trainable parameters: {trainable_params}')
print(f'Non-trainable parameters: {non_trainable_params}')

Total parameters: 4019077
Trainable parameters: 11529
Non-trainable parameters: 4007548


In [9]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

device = torch.device("cpu")
if torch.cuda.is_available():
	device = torch.device("cuda")
elif torch.backends.mps.is_available():
	device = torch.device("mps")
else:
	device = torch.device("cpu")
print(device)
	

mps


In [10]:
from tqdm import tqdm

num_epochs = 20
model.to(device)
running_loss = []
val_loss = []
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # Wrap your data loader with tqdm for a progress bar
    print('Learning rate:', scheduler.get_last_lr())
    train_loader = tqdm(train_loader)

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss.append(loss.item())
                
        # Update the progress bar with the loss information
        train_loader.set_description(f'Epoch {epoch+1}/{num_epochs}')
        train_loader.set_postfix(loss=loss.item())
    scheduler.step()
    

with torch.no_grad():
    model.eval()
    val_loss = 0.0
    val_loader = tqdm(val_loader)
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss.append(loss.item())

        val_loader.set_description(f'Validation Epoch {epoch+1}/{num_epochs}')
        val_loader.set_postfix(loss=loss.item())
torch.save(model.state_dict(), 'model_weights.pth')



Learning rate: [0.01]


Epoch 1/20: 100%|██████████| 119/119 [00:42<00:00,  2.80it/s, loss=0.646]


Learning rate: [0.0095]


Epoch 2/20: 100%|██████████| 119/119 [00:41<00:00,  2.89it/s, loss=1.23] 


Learning rate: [0.009025]


...

Epoch 18/20: 100%|██████████| 119/119 [00:40<00:00,  2.93it/s, loss=0.184] 


Learning rate: [0.003972143184582182]


Epoch 19/20: 100%|██████████| 119/119 [00:40<00:00,  2.92it/s, loss=0.118]


Learning rate: [0.0037735360253530726]


Validation Epoch 20/20: 100%|██████████| 30/30 [00:09<00:00,  3.08it/s, loss=0.721]


In [12]:
# plot the loss and accuracy
# running_loss and val_loss are the losses for each epoch



28.811276875436306


# load file to test

In [20]:
from PIL import Image
from tkinter import Tk
from tkinter.filedialog import askopenfilename


In [34]:
Tk().withdraw() # Prevents an empty tkinter window from appearing
image_path = askopenfilename() # Opens a dialog to choose an image

img = Image.open(image_path).convert('RGB')
img_tensor = data_transforms(img).unsqueeze(0) # Add batch dimension

with torch.no_grad(): # Disable gradient computation for inference
    img_tensor = img_tensor.to(device)
    output = model(img_tensor)
    print(output)  # Example output
    # Assuming a single-label classification, get the predicted class
    _, predicted_class = torch.max(output, 1)
    
    print(f'Predicted class: {predicted_class.item()}') # Example output
    print(f'Predicted class: {train_dataset.dataset.classes[predicted_class.item()]}') # Example output


tensor([[ 1.3930,  0.3082, -4.8347, -7.2462,  3.9851,  1.2308, -3.5068, -1.5247,
         -2.6965]], device='mps:0')
Predicted class: 4
Predicted class: Miscellaneous Trash


In [25]:
print(train_dataset.dataset.classes)

['Cardboard', 'Food Organics', 'Glass', 'Metal', 'Miscellaneous Trash', 'Paper', 'Plastic', 'Textile Trash', 'Vegetation']
